In [16]:
from pypdf import PdfReader
from langchain_huggingface import HuggingFaceEmbeddings
# from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from pprint import pprint
from langchain_groq import ChatGroq
import os
# from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.llms import HuggingFaceTextGenInference
from langchain_huggingface import ChatHuggingFace
# import huggingface_hub
# from huggingface_hub import whoami
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document

In [2]:
# Read the API key from a file
with open("C:\Programming\doc reader chatbot\groq.txt", "r") as file:
    apiKey = file.read().strip()
# Set the API key as an environment variable
os.environ["GROQ_API_KEY"] = apiKey

<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\mmibr\AppData\Local\Temp\ipykernel_2252\3610170531.py:2: SyntaxWarning: invalid escape sequence '\P'
  with open("C:\Programming\doc reader chatbot\groq.txt", "r") as file:


In [41]:
def processText(text):
    # Split the text into chunks using Langchain's RecursiveCharacterTextSplitter
    textSplitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 50,
        # length_function=len
        add_start_index=True
    )
    print(type(textSplitter))
    print(textSplitter)
    chunks = textSplitter.split_documents(text)
    print(type(chunks))
    print(len(chunks))
    # print(len(chunks[0]))
    print(type(chunks[0]))
    print(chunks[0])
    print(chunks[1])
    for chunk in chunks:
        print(f"Chunk metadata: {chunk.metadata}")
        print(f"Chunk length: {len(chunk.page_content)}")
    print(f"Split {len(text)} documents into {len(chunks)} chunks.")

    # # Convert the chunks of text into embeddings to form a database
    # embeddings = HuggingFaceEmbeddings()
    # db = Chroma.from_texts(chunks, embeddings)

    # return db

In [4]:
# TODO: Understand this function fully
def analyzeResults(query, results):
    prompt_template = """
    Answer the query based only on the given context:
    {context}
    ---
    Answer the query based on the above context: {query}
    """

    context = "\n\n---\n\n".join([doc.page_content for doc, _score in results])

    prompt = prompt_template.format(context = context, query = query)

    llm = ChatGroq()
    # llm = ChatGroq(model="llama3-8b-8192")
    # llm = ChatGroq(model = "llama-3.1-70b-versatile")
    response = llm.invoke(prompt)

    return response

In [37]:
file = "the-inspector-general--2.pdf"
# pdf = PdfReader(file)
loaderfile = PyPDFLoader(file)
docs = loaderfile.load()
# print(len(pdf.pages))
print(type(file))
# print(type(pdf))
print(type(loaderfile))
print(type(docs))
print()

for doc in docs:
    print(type(doc))
    print(doc.metadata)
    print(type(doc.metadata))
    print(len(doc.metadata))
    # print(type(doc.page_content))
    print(len(doc.page_content))
    print()

<class 'str'>
<class 'langchain_community.document_loaders.pdf.PyPDFLoader'>
<class 'list'>

<class 'langchain_core.documents.base.Document'>
{'source': 'the-inspector-general--2.pdf', 'page': 0}
<class 'dict'>
2
73

<class 'langchain_core.documents.base.Document'>
{'source': 'the-inspector-general--2.pdf', 'page': 1}
<class 'dict'>
2
999

<class 'langchain_core.documents.base.Document'>
{'source': 'the-inspector-general--2.pdf', 'page': 2}
<class 'dict'>
2
2547

<class 'langchain_core.documents.base.Document'>
{'source': 'the-inspector-general--2.pdf', 'page': 3}
<class 'dict'>
2
1662

<class 'langchain_core.documents.base.Document'>
{'source': 'the-inspector-general--2.pdf', 'page': 4}
<class 'dict'>
2
2643

<class 'langchain_core.documents.base.Document'>
{'source': 'the-inspector-general--2.pdf', 'page': 5}
<class 'dict'>
2
2581

<class 'langchain_core.documents.base.Document'>
{'source': 'the-inspector-general--2.pdf', 'page': 6}
<class 'dict'>
2
2559

<class 'langchain_core.docum

In [40]:
# text = ""
# for page in pdf.pages:
#     text += page.extract_text()
# print(len(text))

In [42]:
db = processText(docs)

<class 'langchain_text_splitters.character.RecursiveCharacterTextSplitter'>
<class 'list'>
188
<class 'langchain_core.documents.base.Document'>
page_content='The	Inspector-General
	
By
	
Nikolái	Vasí
lievi
ch	Gó
gol' metadata={'source': 'the-inspector-general--2.pdf', 'page': 0, 'start_index': 8}
page_content='CHARACTERS	OF	THE	PLAY
	
ANTON	ANTONOVICH	SKVOZNIK-DMUKHANOVSKY,	the
	
Governor.
ANNA	ANDREYEVNA,	his	wife.
MARYA	ANTONOVNA,	his	daughter.
LUKA	LUKICH	KHLOPOV,	the	Inspector	of	Schools.
His	Wife.
AMMOS	FIODOROVICH	LIAPKIN-TIAPKIN,	the	Judge.
ARTEMY	FILIPPOVICH	ZEMLIANIKA,	the	Superintendent	of
	
Charities.
IVAN	KUZMICH	SHPEKIN,	the	Postmaster.
PIOTR	IVANOVICH	DOBCHINSKY.	}
PIOTR	IVANOVICH	BOBCHINSKY.	}	Country	Squires.
IVAN	ALEKSANDROVICH	KHLESTAKOV,	an	official	from	St.
	
Petersburg.
OSIP,	his	servant.
CHRISTIAN	IVANOVICH	HÜBNER,	the	district	Doctor.
FIODR	ANDREYEVICH	LULIUKOV.	ex-officials,
IVAN	LAZAREVICH	RASTAKOVSKY.	personages
STEPAN	IVANOVICH	KOROBKIN.	of	the	town.
STEPAN	I

In [69]:
query = "What is the premise of the story?"

In [115]:
results = db.similarity_search_with_score(query)
pprint(results)

[(Document(page_content="ANNA.\tWhere?\tJust\tyour\timagination\tagain!—Why,\tyes,\tsomeone\tiscoming.\tI\twonder\twho\tit\tis.\tA\tshort\tman\tin\ta\tfrock\tcoat.\tWho\tcan\tit\tbe?\tEh?\nThe\tsuspense\tis\tawful!\tWho\tcan\tit\tbe,\tI\twonder.\nMARYA.\tDobchinsky,\tmamma.\nANNA.\tDobchinsky!\tYour\timagination\tagain!\tIt's\tnot\tDobchinsky\tat\tall.\n[Waves\ther\thandkerchief.]\tHo,\tyou!\tCome\there!\tQuick!\nMARYA.\tIt\tis\tDobchinsky,\tmamma.\nANNA.\tOf\tcourse,\tyou've\tgot\tto\tcontradict.\tI\ttell\tyou,\tit's\tnot\tDobchinsky.\nMARYA.\tWell,\twell,\tmamma?\tIsn't\tit\tDobchinsky?\nANNA.\tYes,\tit\tis,\tI\tsee\tnow.\tWhy\tdo\tyou\targue\tabout\tit?\t[Calls\tthrough\tthe\nwindow.]\tHurry\tup,\tquick!\tYou're\tso\tslow.\tWell,\twhere\tare\tthey?\tWhat?\tSpeak\nfrom\twhere\tyou\tare.\tIt's\tall\tthe\tsame.\tWhat?\tHe\tis\tvery\tstrict?\tEh?\tAnd\thow\nabout\tmy\thusband?\t[Moves\taway\ta\tlittle\tfrom\tthe\twindow,\texasperated.]\tHe\tis\nso\tstupid.\tHe\twon't\tsay\ta\tword\tunti

In [116]:
response = analyzeResults(query, results)

In [117]:
pprint(response)

AIMessage(content="Based on the given context, the story appears to be set in a town where someone important is expected to arrive. The characters, Anna and Marya, are waiting near a window for this person to come. Anna expresses her frustration as she believes that the town is dirty and unkempt. She also mentions that she had relied on Dobchinsky but is now feeling let down.\n\nWhen Marya suggests that Dobchinsky is the person approaching, Anna initially disagrees but then realizes that it is indeed him. Anna calls Dobchinsky to hurry up and expresses her annoyance at his slow pace.\n\nScene II begins with Dobchinsky entering. Anna reprimands him for not being diligent and tells him that she had relied on him to take care of things.\n\nBased on this context, the story appears to be a critique of the town's lack of cleanliness and the inefficiency of its officials. The main character, Anna, is frustrated with the way things are being handled and expresses her disappointment with Dobchi

In [118]:
sources = [doc.metadata.get("id", None) for doc, _score in results]
formattedResponse = f"Response: {response}\nSources: {sources}"
pprint(formattedResponse)

('Response: content="Based on the given context, the story appears to be set '
 'in a town where someone important is expected to arrive. The characters, '
 'Anna and Marya, are waiting near a window for this person to come. Anna '
 'expresses her frustration as she believes that the town is dirty and '
 'unkempt. She also mentions that she had relied on Dobchinsky but is now '
 'feeling let down.\\n\\nWhen Marya suggests that Dobchinsky is the person '
 'approaching, Anna initially disagrees but then realizes that it is indeed '
 'him. Anna calls Dobchinsky to hurry up and expresses her annoyance at his '
 'slow pace.\\n\\nScene II begins with Dobchinsky entering. Anna reprimands '
 'him for not being diligent and tells him that she had relied on him to take '
 'care of things.\\n\\nBased on this context, the story appears to be a '
 "critique of the town's lack of cleanliness and the inefficiency of its "
 'officials. The main character, Anna, is frustrated with the way things are '
